<a href="https://colab.research.google.com/github/Prishita17/Movie-Recommendation-System/blob/main/Optimised_Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision matplotlib

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [3]:
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)


100%|██████████| 9.91M/9.91M [00:00<00:00, 18.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 490kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.67MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.82MB/s]


In [4]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)   # Conv layer
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)  # Conv layer
        self.fc1 = nn.Linear(9216, 128)               # Fully connected
        self.fc2 = nn.Linear(128, 10)                 # Output layer (10 digits)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
model = SimpleCNN()

In [6]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [7]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data, target
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx*len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}")


In [8]:
def test(model, test_loader):
    model.eval()
    correct = 0
    test_loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f"Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)")
    return accuracy

In [9]:
for epoch in range(1, 2):  # keep 1 epoch for speed; increase for better results
    train(model, "cpu", train_loader, optimizer, epoch)
    test(model, test_loader)

Train Epoch: 1 [0/60000] Loss: 2.310826
Train Epoch: 1 [6400/60000] Loss: 0.251034
Train Epoch: 1 [12800/60000] Loss: 0.249806
Train Epoch: 1 [19200/60000] Loss: 0.067251
Train Epoch: 1 [25600/60000] Loss: 0.033897
Train Epoch: 1 [32000/60000] Loss: 0.083742
Train Epoch: 1 [38400/60000] Loss: 0.072199
Train Epoch: 1 [44800/60000] Loss: 0.017842
Train Epoch: 1 [51200/60000] Loss: 0.058101
Train Epoch: 1 [57600/60000] Loss: 0.005716
Test set: Average loss: 0.0001, Accuracy: 9825/10000 (98.25%)


In [10]:
torch.save(model.state_dict(), "mnist_cnn.pth")
print("✅ Model saved as mnist_cnn.pth")

✅ Model saved as mnist_cnn.pth
